In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
week16_link = 'http://www.footballlocks.com/nfl_odds_week_16.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week16_html = requests.get(week16_link).text

soup = BeautifulSoup(week16_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[277])

<tr>
<td>12/23 4:30 ET</td>
<td>At Baltimore</td>
<td>-13.5</td>
<td>Indianapolis</td>
<td>41</td>
<td>-$1000 +$700</td>
</tr>
<tr>
<td>12/25 8:30 ET</td>
<td>At Miami</td>
<td>-2</td>
<td>NY Jets</td>
<td>36.5</td>
<td>-$140 +$120</td>
</tr>


In [4]:
#want to 278 because it does not include last one
table = tr[51:278]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week16_df = pd.DataFrame(string_table)

week16_df.columns = ['all_data']

week16_df.head()

,all_data
0,<tr>\n<td>12/23 4:30 ET</td>\n<td>At Baltimore...
1,<tr>\n<td>12/23 8:30 ET</td>\n<td>Minnesota</t...
2,<tr>\n<td>12/24 1:00 ET</td>\n<td>Detroit</td>...
3,<tr>\n<td>12/24 1:00 ET</td>\n<td>LA Chargers<...
4,<tr>\n<td>12/24 1:00 ET</td>\n<td>LA Rams</td>...


In [5]:
#want to clean the date data

dates = [x.split('td')[1] for x in week16_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week16_df['date'] = cleaned_dates

In [6]:
#pull out favorites

week16_df['cleaned_favorites'] = week16_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week16_df['cleaned_underdogs'] = week16_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week16_df['cleaned_spread'] = week16_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week16_df['cleaned_over_under'] = week16_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week16_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week16_df['money_odds'] = money_odds

#input week number for each game
week16_df['week_number'] = '16'

In [7]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2018
year_list = []
for x in week16_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week16_df['year'] = year_list

In [8]:
#create a final dataframe for week 16
week16_final = week16_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week16_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,12/23 4:30 ET,2018,16,At Baltimore,Indianapolis,-13.5,41,-$1000 +$700
1,12/23 8:30 ET,2018,16,Minnesota,At Green Bay,-9,41,-$450 +$350
2,12/24 1:00 ET,2018,16,Detroit,At Cincinnati,-3,43.5,-$175 +$155
3,12/24 1:00 ET,2018,16,LA Chargers,At NY Jets,-7,42.5,-$330 +$270
4,12/24 1:00 ET,2018,16,LA Rams,At Tennessee,-6,46,-$250 +$210


In [9]:
#week16_final to pickle
week16_final.to_pickle('week16_final.pkl')